In [ ]:
class Model:
    
    def bow_embedding(df):
    from nltk.tokenize import RegexpTokenizer
    from sklearn.feature_extraction.text import TfidfVectorizer


    token=RegexpTokenizer(r'[a-zA-Z0-9]+')
    tfidf=TfidfVectorizer(stop_words='english',tokenizer=token.tokenize)
    text=tfidf.fit_transform(clean_df['lemma'])
    
    
    def model_class_imbalance(label,threshold,num_classes=2):
        c=[]

        l=len(label)/num_classes
        print("**********************************")
        print("Ideal class size : ",l)
        for i in range(num_classes):

            k=label[label==4*i].count()
            c.append(k)

            print("Length of class",i,": ",c[i])
            if (1-threshold)*l<c[i]<(1+threshold)*l:
                continue;

            else:
                print('There exists a Class Imbalance in your Dataset')
                return True
                break;
        return False
        
    def test_train(vectorized_matrix=text,label=clean_df['Polarity']):
        x_train,x_test,y_train,y_test=train_test_split(vectorized_matrix,label,test_size=0.2,shuffle=True)
        while model_class_imbalance(y_train,threshold=0.05) or model_class_imbalance(y_test,0.05)==True:
            x_train,x_test,y_train,y_test=train_test_split(text,clean_df['Polarity'],test_size=0.2,shuffle=True)
        return x_train,x_test,y_train,y_test
    
    def naive_bayes(x_train,x_test,y_train,y_test):
        from sklearn import metrics
        def multinomialNB(x_train,x_test,y_train,y_test):
            clf1=MultinomialNB()
            %time clf1.fit(x_train,y_train)
            ypred1=clf1.predict(x_test)
            print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, ypred1))
            
        def bernoulliNB(x_train,x_test,y_train,y_test):
            %time clf3 = BernoulliNB().fit(x_train,y_train)
            ypred3= clf3.predict(x_test)
            print("BernoulliNB Accuracy:",metrics.accuracy_score(y_test,ypred3))
            
    def Deep_network(df):
        from tensorflow.keras.preprocessing.text import Tokenizer
        from tensorflow.keras.preprocessing.sequence import pad_sequences
        from tensorflow.keras.layers import Embedding,SpatialDropout1D,LSTM,Dense
        from sklearn.metrics import f1_score as f1
        token=Tokenizer(lower=True,filters='')
        token.fit_on_texts(full_text)
        max_len=37
        train_tokenized=token.texts_to_sequences(df)
        X=pad_sequences(train_tokenized,maxlen=max_len)
        x_train,x_test,y_train,y_test=test_train(X,label)
        print(" shape of x train and y train: ",x_train.shape,y_train.shape)
        print(" shape of x test and y test: ",x_test.shape,y_test.shape)
        
        
        embed_dim=150,
        lstm_out=200
        max_features=X.max()+1
        lr=0.001
        warmup_epoch=20
        model=tensorflow.keras.Sequential([
            Embedding(max_features,embed_dim,input_length=X.shape[1]),
            SpatialDropout1D(0.3),
            LSTM(lstm_out,dropout=0.2,recurrent_dropout=0.2),
            Dense(1,activation='sigmoid')           
        ])
        
        model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy',f1])
        